## Initial Lib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

## Keypoint Mediapipe

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

## Dataset

In [3]:
actions = np.array(['F', 'I', 'J', 'K', 'Z']) #

DATA_PATH = (r"C:\Users\victus\Documents\SignLanguage\MotionDetectionforSIBIv2\Dataset") 

no_sequences = 30

sequence_length = 30

start_folder = 30

In [4]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'F': 0, 'I': 1, 'J': 2, 'K': 3, 'Z': 4}

## Train Test

In [5]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), '{}_keypoints_{}.npy'.format(action, frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [6]:
np.array(sequences).shape

(150, 30, 258)

In [7]:
np.array(labels).shape

(150,)

In [8]:
X = np.array(sequences)

In [9]:
X.shape

(150, 30, 258)

In [10]:
y = to_categorical(labels).astype(int)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [12]:
y_test.shape

(8, 5)

## Build Model

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import  Input, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import Callback

In [14]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [15]:
class StopTrainingCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch > 150 and logs.get('loss') < 0.09 and (logs.get('categorical_accuracy') < 0.99 and logs.get('categorical_accuracy') > 0.88):
            print("Stopping training")
            self.model.stop_training = True
            
stop_callback = StopTrainingCallback()

In [16]:
lstm_input = Input(shape=(30, 258))

In [60]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [61]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [51]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback, stop_callback]) #

Epoch 1/500
5/5 [==============================] - 2s 26ms/step - loss: 1.6087 - categorical_accuracy: 0.2535
Epoch 2/500
5/5 [==============================] - 0s 27ms/step - loss: 1.5448 - categorical_accuracy: 0.2958
Epoch 3/500
5/5 [==============================] - 0s 26ms/step - loss: 1.3962 - categorical_accuracy: 0.4155
Epoch 4/500
5/5 [==============================] - 0s 26ms/step - loss: 1.0653 - categorical_accuracy: 0.5634
Epoch 5/500
5/5 [==============================] - 0s 25ms/step - loss: 0.9333 - categorical_accuracy: 0.5775
Epoch 6/500
5/5 [==============================] - 0s 25ms/step - loss: 0.9634 - categorical_accuracy: 0.5423
Epoch 7/500
5/5 [==============================] - 0s 25ms/step - loss: 0.7920 - categorical_accuracy: 0.6549
Epoch 8/500
5/5 [==============================] - 0s 25ms/step - loss: 0.6001 - categorical_accuracy: 0.6972
Epoch 9/500
5/5 [==============================] - 0s 27ms/step - loss: 0.5434 - categorical_accuracy: 0.7254
Epoch 10/5

5/5 [==============================] - 0s 25ms/step - loss: 0.2757 - categorical_accuracy: 0.8592
Epoch 75/500
5/5 [==============================] - 0s 26ms/step - loss: 0.3017 - categorical_accuracy: 0.8592
Epoch 76/500
5/5 [==============================] - 0s 26ms/step - loss: 0.3744 - categorical_accuracy: 0.8521
Epoch 77/500
5/5 [==============================] - 0s 26ms/step - loss: 0.3455 - categorical_accuracy: 0.8451
Epoch 78/500
5/5 [==============================] - 0s 26ms/step - loss: 0.2874 - categorical_accuracy: 0.8592
Epoch 79/500
5/5 [==============================] - 0s 28ms/step - loss: 0.2859 - categorical_accuracy: 0.8732
Epoch 80/500
5/5 [==============================] - 0s 27ms/step - loss: 0.2726 - categorical_accuracy: 0.8451
Epoch 81/500
5/5 [==============================] - 0s 26ms/step - loss: 0.2215 - categorical_accuracy: 0.9437
Epoch 82/500
5/5 [==============================] - 0s 26ms/step - loss: 0.2349 - categorical_accuracy: 0.8873
Epoch 83/500
5

5/5 [==============================] - 0s 25ms/step - loss: 0.1705 - categorical_accuracy: 0.9225
Epoch 148/500
5/5 [==============================] - 0s 25ms/step - loss: 0.2549 - categorical_accuracy: 0.8803
Epoch 149/500
5/5 [==============================] - 0s 25ms/step - loss: 0.3099 - categorical_accuracy: 0.8662
Epoch 150/500
5/5 [==============================] - 0s 27ms/step - loss: 0.1795 - categorical_accuracy: 0.9225
Epoch 151/500
5/5 [==============================] - 0s 26ms/step - loss: 0.1887 - categorical_accuracy: 0.9225
Epoch 152/500
5/5 [==============================] - 0s 26ms/step - loss: 0.1921 - categorical_accuracy: 0.9225
Epoch 153/500
5/5 [==============================] - 0s 26ms/step - loss: 0.1958 - categorical_accuracy: 0.9014
Epoch 154/500
5/5 [==============================] - 0s 26ms/step - loss: 0.1595 - categorical_accuracy: 0.9366
Epoch 155/500
5/5 [==============================] - 0s 26ms/step - loss: 0.1632 - categorical_accuracy: 0.9437
Epoch 

In [52]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 30, 64)            82688     
                                                                 
 lstm_13 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_14 (LSTM)              (None, 64)                49408     
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 5)                 165       
                                                                 
Total params: 237,317
Trainable params: 237,317
Non-tr

## Matrix and Accuracy

In [53]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [54]:
yhat = model.predict(X_test)

In [55]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [56]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[6, 0],
        [0, 2]],

       [[6, 0],
        [0, 2]],

       [[7, 0],
        [0, 1]],

       [[5, 0],
        [0, 3]]], dtype=int64)

In [57]:
accuracy_score(ytrue, yhat)

1.0

## Save and Reload Model

In [58]:
model.save('abjadV1.h5')

In [59]:
del model

In [62]:
model.load_weights('abjadV1.h5')

## Real Time test

In [64]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.3) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

        height, width, channels = image.shape
        text_size, _ = cv2.getTextSize(' '.join(sentence), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        text_width = text_size[0]
        x = (width - text_width) // 2
        cv2.putText(image, ' '.join(sentence), (x, height - 40), 
        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I
<class 'mediapipe.python.solution_base.SolutionOutputs

<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>

Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Z
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs'>
J
<class 'mediapipe.python.solution_base.SolutionOutputs